### Mushroom

In [153]:
import pandas as pd
import numpy as np


# Note that I added one line to this file, namely the names from the dataset attribute information.
data = pd.read_csv("results.csv")

In [154]:
# sort the data
data_sorted = data.copy()
counter = 0
for k in range(3):
    for i in range(48):
        for j in range(5):
            data_sorted.iloc[counter] = data.iloc[(48*j + i)+k*240]
            counter += 1

In [166]:
# Take the mean of experiments
med_data = data_sorted.values[:,1:]
new_data = []
for i in range(int(len(med_data)/5)):
    new_data.append([med_data[i*5, 0], med_data[i*5, 1], med_data[i*5, 2], med_data[i*5, 3], med_data[i*5, 4], med_data[i*5, 5], (med_data[int(5*i+0), 7]+med_data[int(5*i+1), 7]+med_data[int(5*i+2), 7]+med_data[int(5*i+3), 7]+med_data[int(5*i+4),7])/5])
len(new_data)


144

In [167]:
new_data = np.asarray(new_data, dtype=object)


In [168]:
res_data = []
for i in range(int(len(new_data)/3)):
    bestSetup = ""
    if (new_data[int(i+0*len(new_data)/3), 6] > new_data[int(i+1*len(new_data)/3), 6]):
        if new_data[int(i+1*len(new_data)/3), 6] > new_data[int(i+2*len(new_data)/3), 6]:
            bestSetup = "optimized"
        else:
            bestSetup = "hybrid"
    else:
        if new_data[int(i+0*len(new_data)/3), 6] > new_data[int(i+2*len(new_data)/3), 6]:
            bestSetup = "optimized"
        else:
            bestSetup = "standard"
    res_data.append([new_data[i, 1], new_data[i, 2], new_data[i, 3], new_data[i, 4], new_data[i, 5], bestSetup])

In [169]:
res_data_cost_based_inter = []
for i in range(int(len(new_data)/3)):
    res_data_cost_based_inter.append(np.append(new_data[int(i+0*len(new_data)/3),:6],new_data[int(i+0*len(new_data)/3),6]*0.194236/3600000))
    res_data_cost_based_inter.append(np.append(new_data[int(i+1*len(new_data)/3),:6],new_data[int(i+1*len(new_data)/3),6]*(0.2088+0.194236)/2/3600000))
    res_data_cost_based_inter.append(np.append(new_data[int(i+2*len(new_data)/3),:6],new_data[int(i+2*len(new_data)/3),6]*0.2088/3600000))

In [170]:
res_data_cost_based_inter = np.asarray(res_data_cost_based_inter, dtype=object)

In [171]:
dfcost = pd.DataFrame(res_data_cost_based_inter)

In [172]:
res_data_cost_based = []
for i in range(int(len(res_data_cost_based_inter)/3)):
    bestSetup = ""
    if (res_data_cost_based_inter[int(3*i+0), 6] > res_data_cost_based_inter[int(3*i+1), 6]):
        if res_data_cost_based_inter[int(3*i+1), 6] > res_data_cost_based_inter[int(3*i+2), 6]:
            bestSetup = res_data_cost_based_inter[int(3*i+2), 0]
        else:
            bestSetup = res_data_cost_based_inter[int(3*i+1), 0]
    else:
        if res_data_cost_based_inter[int(3*i+0), 6] > res_data_cost_based_inter[int(3*i+2), 6]:
            bestSetup = res_data_cost_based_inter[int(3*i+2), 0]
        else:
            bestSetup = res_data_cost_based_inter[int(3*i+0), 0]
    res_data_cost_based.append([res_data_cost_based_inter[3*i, 1], res_data_cost_based_inter[i, 2], res_data_cost_based_inter[i, 3], res_data_cost_based_inter[i, 4], res_data_cost_based_inter[i, 5], bestSetup])

In [173]:
df = pd.DataFrame(res_data_cost_based)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le.fit(['standard', 'hybrid', 'optimized'])
le2.fit(['bi-rnn.py', 'lenet5.py'])
df[5] = le.transform(df[5])
df[4] = le2.transform(df[4])


In [174]:
import sklearn.preprocessing as pp
# data = data.apply(pp.LabelEncoder().fit_transform)
X = df.iloc[:,:-1]
y = df.iloc[:, 5]
for col in X.columns:
    X[col].astype('category')

In [177]:
svmN = 0
svmV = 0
knnN = 0
knnV = 0
mlpN = 0
mlpV = 0
lrN = 0
lrV = 0
lr2N = 0
lr2V = 0
ldV = 0
ldN = 0
gnbN = 0
gnbV = 0
dtcN = 0
dtcV = 0
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
    
#     DATA IS SCALED, SHOULD ALSO BE SCALED FOR INPUT!
    scaler = preprocessing.StandardScaler().fit(x_train)
    X_train_scaled = scaler.transform(x_train)
    X_test_scaled = scaler.transform(x_test)


    mlp_clf = MLPClassifier(max_iter=2000, solver="sgd")
    mlp_clf.fit(X_train_scaled, y_train)
    y_pred = mlp_clf.predict(X_test_scaled)
    mlpV +=accuracy_score(y_test, y_pred)*100
    mlpN +=1
    print("MLP Accuracy: " + str(mlpV/mlpN) + "%")


MLP Accuracy: 100.0%
MLP Accuracy: 100.0%
MLP Accuracy: 93.33333333333333%
MLP Accuracy: 90.0%
MLP Accuracy: 92.0%
MLP Accuracy: 93.33333333333333%
MLP Accuracy: 91.42857142857143%
MLP Accuracy: 92.5%
MLP Accuracy: 91.11111111111111%
MLP Accuracy: 90.0%


In [178]:
dfscaled = pd.DataFrame(X_train_scaled)
dfscaled2 = pd.DataFrame()
dfscaled2['batchsize']=dfscaled[0]
dfscaled2['maxepochs']=dfscaled[1]
dfscaled2['learningrate']=dfscaled[2]
dfscaled2['memorysize']=dfscaled[3]
dfscaled2['model']=dfscaled[4]
dfscaled2['setup']=y_train.values

In [179]:
# batch_size	max_epochs	learning_rate	memory_size	model
df2 = pd.read_csv("design_final.csv")[:30]
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['bi-rnn.py', 'lenet5.py'])
df2['model'] = le.transform(df2['model'])
df2 = df2.drop(df2.columns[0], axis=1)

mlpPred = mlp_clf.predict(scaler.transform(df2))
print("["+','.join(map(str, mlpPred)) + "]")



[0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0]
